In [ ]:
import torch
from hydra import compose, initialize
from PIL import Image 

# Ensure that the necessary repository is cloned and installed. You may need to run: 
# git clone git@github.com:WongKinYiu/YOLO.git
# cd YOLO 
# pip install .
from yolo import AugmentationComposer, bbox_nms, Config, create_model, custom_logger, draw_bboxes, Vec2Box

In [ ]:
CONFIG_PATH = "../yolo/config"
CONFIG_NAME = "config"

DEVICE = 'cpu'
CLASS_NUM = 80
WEIGHT_PATH = '../weights/v9-c-deploy.pt' 
IMAGE_PATH = '../demo/images/inference/image.png'

custom_logger()
device = torch.device(DEVICE)
image = Image.open(IMAGE_PATH)

In [ ]:
with initialize(config_path=CONFIG_PATH, version_base=None, job_name="notebook_job"):
    cfg: Config = compose(config_name=CONFIG_NAME, overrides=["task=inference", f"task.data.source={IMAGE_PATH}", "model=v9-c-deploy"])
    model = create_model(cfg.model, class_num=CLASS_NUM, weight_path=WEIGHT_PATH, device=device)
    transform = AugmentationComposer([], cfg.image_size)
    vec2box = Vec2Box(model, cfg.image_size, device)

In [ ]:
image, bbox = transform(image, torch.zeros(0, 5))
image = image.to(device)[None]

In [ ]:
with torch.no_grad():
    predict = model(image)
    predict = vec2box(predict["Main"])

predict_box = bbox_nms(predict[0], predict[2], cfg.task.nms)
draw_bboxes(image, predict_box, idx2label=cfg.class_list)

Sample Output:

![image](../demo/images/output/visualize.png)